In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
import online_nmf

In [2]:
n_ratings = 100000
latent_factors = 2
n_iterations = 5
data, model, movies, R_df, test, users = online_nmf.training(n_ratings, latent_factors, n_iterations)
print("users:",users.shape[0])

('users:', 702)


In [6]:
#M_df = R_df.apply(lambda x: 2*x, axis=1).astype('int32').head()

In [18]:
def get_RMSE():
    from numpy.random import choice
    from sklearn.metrics import mean_squared_error as mse

    N_test = int(users.shape[0]*0.2)
    print "selecting",N_test,"users"
    RMSE = 0.
    RMSEII = 0.
    numbers = choice(users.shape[0], size=N_test)
    reals = []
    estimates = []
    for user_idx in numbers:
        
        real_user = R_df.iloc[user_idx]
        real_user = real_user.values.reshape(-1,real_user.shape[0])[0]
        reals.append(real_user)
        
        est_user = users[user_idx]
        est_user = est_user.reshape(-1,est_user.shape[0])
        est_user = np.rint(model.inverse_transform(est_user))[0]
        estimates.append(est_user)
        
        # # this is the same that sklearn does. Just in case
        #aux = 0.
        #for j in range(len(real_user)):
        #    aux += (real_user[j]-est_user[j])*(real_user[j]-est_user[j])
        #aux = aux/float(len(real_user))
        #RMSEII += aux
        
    RMSE = np.sqrt( mse(reals,estimates) )
    #RMSEII = np.sqrt( RMSEII/float(len(numbers)) )
    
    return RMSE, RMSEII
#get_RMSE()

selecting 140 users


(0.4302174099227036, 0.43021740992270363)

In [52]:
def split_set(R_df):
    import pandas as pd
    from numpy.random import choice
    # users_list = data['userId'].unique()
    train_df = R_df
#    test_df = pd.DataFrame({'userId' : [], 'movieId' : [], 'rating' : []})
    
    print len(R_df.columns)
#    test_movies = choice(R_df.columns, size=int(data.shape[1]*.02), replace=False)
    test_movies = choice(R_df.columns, size=15, replace=False)
    test_ratings = []
    test_users = []
    
    for tm in test_movies:
        aux = R_df[tm].nonzero()[0]
        if aux == []:
            continue
        fm = R_df[tm].nonzero()[0][0] # Movie
#        test_users.append(R_df.index[fm]) # user (got from its index, just in case)
        test_users.append(R_df.index.get_loc(R_df.index[fm]))
        test_ratings.append( R_df[tm][R_df.index[fm]] ) # column = Movie, row = index of first nonzero rating for Movie
        train_df[tm][R_df.index[fm]] = 0
        
    # data_df has some erased entries
    # R_df is the same as before
    test_movies = [ R_df.columns.get_loc(k) for k in test_movies]
    print test_ratings


#    for i in last_rows_idx:
#        test_df = test_df.append(data.loc[i])
#        train_df.drop(i, axis=0, inplace=True)

    #test_df.reset_index(inplace=True)
    #train_df.reset_index(inplace=True)
    #test_df.drop('index',axis=1,inplace=True)
    #train_df.drop('index',axis=1,inplace=True)

    return train_df,test_movies, test_users, test_ratings


In [54]:

def get_RMSEII(users,movies,R_df,test_users,test_movies,test_ratings,model):
    import numpy as np
    #print users[list(test_users)[10]]
    # print(users[347]) just to check
    U = np.asarray(users)
    M = np.transpose(movies)
    test_users = np.array(test_users)
    test_movies = list(test_movies)
    
    estimates = []
    reals = []
    
    RMSEII = 0.
    for idx in range(len(test_users)):
        t = U[test_users[idx]]
        t = t.reshape(-1,t.shape[0])
        tr_usr = model.inverse_transform(t)
        
        l = M[test_movies[idx]]
        l = l.reshape(-1,l.shape[0])
        tr_mov = model.inverse_transform(l)
#        print(tr_mov.shape)
#        print(tr_usr.shape)
        realthing = test_ratings[idx]#R_df.iloc[test_movies[idx]].iloc[test_users[idx]]
        rat = np.dot(tr_mov[0],tr_usr[0])
        RMSEII += (rat-realthing)*(rat-realthing)
        RMSEII = np.sqrt(RMSEII/float(len(test_users)))
    return RMSEII

train_df,test_movies, test_users, test_ratings = split_set(R_df)
get_RMSEII(users,movies,R_df,test_users,test_movies,test_ratings,model)

8227


/home/diego/anaconda3/envs/recom_env/lib/python2.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  app.launch_new_instance()
/home/diego/anaconda3/envs/recom_env/lib/python2.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  app.launch_new_instance()


IndexError: index 0 is out of bounds for axis 0 with size 0